# This file reads in the top libraries contributor networks



In [1]:
import pandas as pd
import bokeh
import glob
import csv
import math
import powerlaw

# Update this cell with your working directory

In [2]:
#Put your path here for unzipped Python_Eco_GHT_data
work_dir = r"C:\Users\thoma\PycharmProjects\GHT_python_data\**\*.csv"

### You have to help clean the data. Go into each commits file and delete the column one past commit_comments (AG is opened in a spreadsheet). If you receive an error just go to the line number and delete the comments that wnet into a new cell.

In [3]:
#Dictionary below gives order of files for each sub_directory of files
#working_list = {"commits":{}, "members":{}, "requests":{}, "actions":{}}
#logins = []
working = []
#read in csv files, comments with quotations in them are problematic
for filepath in glob.iglob(work_dir):
    #Skip files without specific repo data
    if "Top_Contributor_Networks" in filepath: 
        pass
    elif "Top_Contributers" in filepath:
        pass
    elif "repo_creation_dates" in filepath: 
        pass
    elif "Python-dateutil" in filepath: 
        pass #not one of top 14
    else: 
        #Still had issues with quotation marks had to change about 7
        print (filepath)
        #Code that was later updated
        #if "logins" in filepath: 
        #    logins.append(pd.read_csv(filepath,low_memory=False).replace('"','', regex=True).replace("'",'', regex=True).replace("`","", regex=True))
        #else: 
        working.append(pd.read_csv(filepath,low_memory=False).replace('"','', regex=True).replace("'",'', regex=True).replace("`","", regex=True))




C:\Users\thoma\PycharmProjects\GHT_python_data\Click\click_commits.csv
C:\Users\thoma\PycharmProjects\GHT_python_data\Click\click_project_members.csv
C:\Users\thoma\PycharmProjects\GHT_python_data\Click\click_pull_requests.csv
C:\Users\thoma\PycharmProjects\GHT_python_data\Click\click_pull_requests_actions.csv
C:\Users\thoma\PycharmProjects\GHT_python_data\CPython\cpython_commits.csv
C:\Users\thoma\PycharmProjects\GHT_python_data\CPython\cpython_project_members.csv
C:\Users\thoma\PycharmProjects\GHT_python_data\CPython\cpython_pull_requests.csv
C:\Users\thoma\PycharmProjects\GHT_python_data\CPython\cpython_pull_requests_actions.csv
C:\Users\thoma\PycharmProjects\GHT_python_data\Django\django_commits.csv
C:\Users\thoma\PycharmProjects\GHT_python_data\Django\django_project_members.csv
C:\Users\thoma\PycharmProjects\GHT_python_data\Django\django_pull_requests.csv
C:\Users\thoma\PycharmProjects\GHT_python_data\Django\django_pull_requests_actions.csv
C:\Users\thoma\PycharmProjects\GHT_pytho

In [4]:
len(working)

60

In [5]:
#breaks into chunks for library files 
working_chuncked = []
count = 0
inter = []
for idx in range(len(working)):
    #print (idx+1, (idx+1)%4)
    if (idx+1)%4 == 0: 
        
        inter.append(working[idx])
        working_chuncked.append(inter)
        inter = []
    else: 
        inter.append(working[idx])

len(working_chuncked)

15

In [6]:
#verify all repos accounted for
repos = ["click", "cpython", "django","matplotlib","numpy","odoo","pandas","pytest",
         "pytest-cov", "pyyaml","requests","scipy","six","sphinx"]
accounted_for = []
for chunk in range(len(working_chuncked)):
    for lib in working_chuncked[chunk]:
        try: 
            if len(lib["repo_name"]) > 0:
                if lib["repo_name"][0] in repos and lib["repo_name"][0] not in accounted_for:
                    accounted_for.append(lib["repo_name"][0])
        except: 
            pass
                
if len(accounted_for) == len(repos):
    print("All accounted for!")
else: 
    print("Something went wrong")

All accounted for!


In [7]:
#Set up datastructure for analysis
'''
Main Dict- nodes: 
    Key: log_in
    Value: Dict
Sub Dict- relationships: 
    Key: "contributor"
    value: list of contributed repos
    Key: "member" 
    value: list of memberships repos
    Key: Commits
    value: # of commits
    Key: "sucessful_requests" (medium link)
    value: List of successful pulls repos
    Key: "failed_requests" (weak link)
    value: List of failed pull repos
    
'''
pax_net = {}

In [14]:
#Helper function to extract all nodes
def get_actors(working_file):
    nodes = []
    #Takes a list of the 4 working files and extracts all unique users
    #returns list of unique author names
    print(working_file[0]['repo_name'][0])
    name_list = ["author_login", "login", "pull_request_author", "actor_login"]
    uniques = []
    #use index of list based on changing names
    #TODO: Could be made more elegent a lot of duplication
    for idx in range(len(working_file)): 
            uniques += list(working_file[idx][name_list[idx]].unique())
            
    #print(uniques)
    for u in uniques:
        if u not in nodes: 
            nodes.append(u)
    
    return nodes

In [15]:
#Create pax net Dictionary as outline above
def create_pax_data2(nodes, working_file):
    #Takes the unique list of nodes (users) and takes the dataframes csv and creates the above dictionary
    for pax in nodes: 
         
            #TODO: add skip clause if in dict
            #for pax in db:
        if pax in pax_net.keys():
            pass
        else: 
            pax_net[pax]={"contributor":[],
                          "failed_contrib":[],
                          "member":[], 
                          "commits": {},
                          "successful_pulls":{},
                          "failed_pulls":{}}
    #Add memberships (working_file[1])
    for idx, row in working_file[1].iterrows():
        if row["repo_name"] not in pax_net[row["login"]]["member"]:
            pax_net[row["login"]]["member"].append(row["repo_name"])
    #Add successful commits (working_file[0])
    for idx, row in working_file[0].iterrows(): 
        #point to specific dictionary of interest
        author = pax_net[row["author_login"]]["commits"]
        #print(author)
        if row["repo_name"] not in repos:
            pass
        else: 
            if row["repo_name"] in author.keys():
                author[row["repo_name"]] += 1
            else: 
                author[row["repo_name"]] = 1
                pax_net[row["author_login"]]["contributor"].append(row["repo_name"])
    #add pull requests
    for idx, row in working_file[2].iterrows():
        #Get pull id and author
        author = row["pull_request_author"]
        pull_id = row["pull_request_id"]
        repo = row["repo_name"]
        #determine if pulled request failed or succeeded
        pull_history = working_file[3].loc[working_file[3]["pull_request_id"]==pull_id]
        #pull = working_file[2].loc[working_file[2]["pull_request_id"]==pull_id]
        #Turn actions into set
        pull_actions = set(pull_history['action'])
        success_merge = {'opened', "merged", "closed"}
        if success_merge.issubset(pull_actions): 
            if repo not in pax_net[author]["contributor"]:
                pax_net[author]["contributor"].append(repo)
            if repo not in pax_net[author]["successful_pulls"].keys():
                pax_net[author]["successful_pulls"][repo] = 1
            else: 
                pax_net[author]["successful_pulls"][repo] += 1
        elif "merged" not in pull_actions and "closed" in pull_actions: 
            if repo not in pax_net[author]["failed_contrib"] and repo not in pax_net[author]["contributor"]:
                pax_net[author]["failed_contrib"].append(repo)
            if repo not in pax_net[author]["failed_pulls"].keys():
                pax_net[author]["failed_pulls"][repo] = 1
            else: 
                pax_net[author]["failed_pulls"][repo] += 1

In [16]:
#Run the previous two functions
#Get full dictionary
for working_file in working_chuncked[0:16]: 
    nodes = get_actors(working_file)    
    create_pax_data2(nodes, working_file)
len(nodes)


click
cpython
django
matplotlib
numpy
odoo
pandas
pytest
pytest-cov
pyyaml
requests
scipy
setuptools
six
sphinx


337

In [17]:
#Save to a serialized object
import pickle

fileObj = open('pax_net3.pkl', 'wb')
pickle.dump(pax_net,fileObj)
fileObj.close()